<a href="https://colab.research.google.com/github/crossmodais/app/blob/main/io.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
# --- Configurações e Constantes ---
DRIVE_MOUNT_PATH = "/content/drive"
GRAPHPATH = os.path.join(DRIVE_MOUNT_PATH, "MyDrive", "graficos")
SUMMARYPATH = os.path.join(DRIVE_MOUNT_PATH, "MyDrive", "sumarios")  # Usando subpasta 'sumarios'
DATASET_URL = "https://docs.google.com/spreadsheets/d/1e8otjghJ3PGvUNnQZevu6bZ0ktjBfWWUXvkRm61_y_8/export?format=csv"
DEFAULT_PALETTE = "viridis"
FIGSIZE = (10, 6)

# Estilo dos gráficos
sns.set(style="darkgrid")
plt.rcParams.update({
    'figure.facecolor': 'black',
    'axes.facecolor': 'black',
    'axes.edgecolor': 'white',
    'axes.labelcolor': 'cyan',
    'xtick.color': 'white',
    'ytick.color': 'white',
    'text.color': 'white',
    'grid.color': 'gray',
    'grid.linestyle': '--',
    'legend.facecolor': 'black',
    'legend.edgecolor': 'white',
    'figure.titlesize': 18,
    'axes.titlesize': 14,
    'axes.labelsize': 12,
})

# --- Funções Auxiliares ---
def save_fig(fig, filename):
    filepath = os.path.join(GRAPHPATH, filename)
    os.makedirs(os.path.dirname(filepath), exist_ok=True)
    fig.savefig(filepath, dpi=300, bbox_inches='tight')
    plt.close(fig)

def load_data_from_url(url):
    try:
        response = requests.get(url)
        response.raise_for_status()
        csv_content = response.content.decode('utf-8')
        df = pd.read_csv(io.StringIO(csv_content))
        return df
    except Exception as e:
        print(f"Erro ao carregar dados: {e}")
        return None

def generate_statistical_summary(df, filename="sumario.txt"):
    try:
        summary = df.describe(include='all').to_string()  # Inclui todas as colunas no sumário
        filepath = os.path.join(SUMMARYPATH, filename)
        os.makedirs(os.path.dirname(filepath), exist_ok=True)
        with open(filepath, 'w') as f:
            f.write(summary)
        print(f"Sumário estatístico salvo em: {filepath}")
    except Exception as e:
        print(f"Erro ao gerar sumário estatístico: {e}")

# --- Função Principal (main) ---
if __name__ == "__main__":
    # Monta o Google Drive
    drive.mount(DRIVE_MOUNT_PATH, force_remount=True)
    print(f"Drive montado em: {DRIVE_MOUNT_PATH}")

    # Cria diretórios para salvar gráficos e sumários
    os.makedirs(GRAPHPATH, exist_ok=True)
    os.makedirs(SUMMARYPATH, exist_ok=True)

    # Carrega os dados
    df = load_data_from_url(DATASET_URL)
    if df is None:
        print("Não foi possível carregar os dados. Encerrando.")
        exit()

    # Verifica as colunas disponíveis no DataFrame
    print("Colunas disponíveis no DataFrame:")
    print(df.columns.tolist())

    # Corrige o nome da coluna 'Regiao' caso esteja escrito de forma diferente
    region_column = None
    for col in df.columns:
        if col.lower() in ['regiao', 'região']:
            region_column = col
            break

    if region_column is None:
        print("Coluna 'Regiao' ou 'Região' não encontrada no DataFrame. Encerrando.")
        exit()

    # Renomeia a coluna para 'Regiao' para padronização
    df.rename(columns={region_column: 'Regiao'}, inplace=True)

    # --- Pré-processamento ---
    # 1. Label Encoding para 'Regiao'
    label_encoder = LabelEncoder()
    df['Regiao_encoded'] = label_encoder.fit_transform(df['Regiao'])

    # 2. Remover colunas com muitos NaNs (se houver)
    for col in df.columns:
        if df[col].isnull().sum() > len(df) * 0.5:  # Mais de 50% NaN
            print(f"Removendo coluna '{col}' devido a alta porcentagem de valores faltantes.")
            df.drop(col, axis=1, inplace=True)

    # 3. Imputar valores faltantes (usando a mediana para numéricos e a moda para categóricos)
    for col in df.columns:
        if df[col].isnull().any():
            if pd.api.types.is_numeric_dtype(df[col]):
                df[col].fillna(df[col].median(), inplace=True)
            else:
                df[col].fillna(df[col].mode()[0], inplace=True)  # [0] para pegar o valor da moda

    # --- Análises e Visualizações ---
    # 1. Heatmap de Correlação (após pré-processamento)
    generate_correlation_heatmap(df, "1. Heatmap de Correlação", "1_heatmap.png")

    # 2. Scatterplots (IDH, Gini, Investimento vs. Ataques e Impacto)
    generate_scatterplot(df, 'IDH', 'Ataques_Ciberneticos(num)', "2. IDH vs. Número de Ataques", "2_scatter_idh_ataques.png", hue='Regiao_encoded')
    generate_scatterplot(df, 'Indice_Gini', 'Ataques_Ciberneticos(num)', "3. Índice de Gini vs. Número de Ataques", "3_scatter_gini_ataques.png", hue='Regiao_encoded')
    generate_scatterplot(df, 'Investimento_Ciberseguranca_per_capita(R$)', 'Impacto_Financeiro_Ataques(R$ Milhoes)', "4. Investimento vs. Impacto Financeiro", "4_scatter_investimento_impacto.png", hue='Regiao_encoded')

    # 3. Histogramas (Distribuições)
    generate_histogram(df, 'Ataques_Ciberneticos(num)', "5. Distribuição do Número de Ataques", "5_hist_ataques.png")
    generate_histogram(df, 'Impacto_Financeiro_Ataques(R$ Milhoes)', "6. Distribuição do Impacto Financeiro", "6_hist_impacto.png")

    # 4. Boxplots (Ataques e Impacto por Região)
    generate_boxplot(df, 'Regiao', 'Ataques_Ciberneticos(num)', "7. Número de Ataques por Região", "7_boxplot_ataques_regiao.png")
    generate_boxplot(df, 'Regiao', 'Impacto_Financeiro_Ataques(R$ Milhoes)', "8. Impacto Financeiro por Região", "8_boxplot_impacto_regiao.png")

    # --- Sumário Estatístico ---
    generate_statistical_summary(df, "sumario.txt")
    print("Análise completa, gráficos e sumário gerados!")

Mounted at /content/drive
Drive montado em: /content/drive
Colunas disponíveis no DataFrame:
['Região', 'IDH', 'Índice de Gini', 'Escolaridade Superior (%)', 'Acesso Banda Larga (%)', 'Investimento Cibersegurança per capita (R)', 'Ataques Cibernéticos (num)', 'Impacto Financeiro Ataques (R Milhões)', 'Legislação Cibersegurança (0-10)', 'Ataques Infraestrutura Crítica (num)']
Colunas 'IDH' ou 'Ataques_Ciberneticos(num)' não encontradas.
Colunas 'Indice_Gini' ou 'Ataques_Ciberneticos(num)' não encontradas.
Colunas 'Investimento_Ciberseguranca_per_capita(R$)' ou 'Impacto_Financeiro_Ataques(R$ Milhoes)' não encontradas.
Coluna 'Ataques_Ciberneticos(num)' não encontrada.
Coluna 'Impacto_Financeiro_Ataques(R$ Milhoes)' não encontrada.
Colunas 'Regiao' ou 'Ataques_Ciberneticos(num)' não encontradas.
Colunas 'Regiao' ou 'Impacto_Financeiro_Ataques(R$ Milhoes)' não encontradas.
Sumário estatístico salvo em: /content/drive/MyDrive/sumarios/sumario.txt
Análise completa, gráficos e sumário gerados